In [1]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from sklearn.dummy import DummyClassifier
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, classification_report
data_train = pd.read_json("processed_training.json")
data_val = pd.read_json("processed_validation.json")

In [2]:
# pip install gensim
# Taggeddocument for training data
taggedDocuments_train = []
for index, data_point_post in data_train.iterrows():
    taggedDocument_train =  TaggedDocument(words=data_train.at[index, 'posts'], tags=[data_train.at[index, 'type']])
    taggedDocuments_train.append(taggedDocument_train)

In [3]:
# Taggeddocument for validation data
taggedDocuments_val = []
for index, data_point_post in data_val.iterrows():
    taggedDocument_val =  TaggedDocument(words=data_val.at[index, 'posts'], tags=[data_val.at[index, 'type']])
    taggedDocuments_val.append(taggedDocument_val)

In [ ]:
# creating the Dov2Vec model
model = Doc2Vec(vector_size=80, min_count=2, epochs=15)
model.build_vocab(taggedDocuments_train)

model.train(taggedDocuments_train, total_examples=model.corpus_count, epochs=model.epochs)

In [5]:
model.save('vector_model.doc2vec')

In [7]:
def data_splitter(model, taggedDocuments):
    """
    infering vectors from the model 
    """
    sents = taggedDocuments[0:45865]
    tags, posts = zip(*[(doc.tags[0], model.infer_vector(doc.words)) for doc in sents])
    return tags, posts

In [46]:
# vectorizing the posts and types
type_train, posts_train = data_splitter(model, taggedDocuments_train)
type_val, posts_val = data_splitter(model, taggedDocuments_val)

In [50]:
# Applying logistic regression

logreg = LogisticRegression(n_jobs=-1, solver='sag')
logreg.fit(posts_train, type_train)

import winsound
winsound.Beep(440, 500)

In [1]:
# Validating the (logreg) model
type_pred = logreg.predict(posts_val)
print(classification_report(type_val, type_pred))

NameError: name 'logreg' is not defined

In [25]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=50)
neigh.fit(posts_train, type_train)
type_pred = neigh.predict(posts_val)
print(classification_report(type_val, type_pred))

              precision    recall  f1-score   support

           E       0.85      0.81      0.83      2298
           I       0.82      0.86      0.84      2273

    accuracy                           0.83      4571
   macro avg       0.83      0.83      0.83      4571
weighted avg       0.83      0.83      0.83      4571



In [4]:
# loading test set NB ONLY USE AFTER BEST HYPERPARAMETER HAS BEEN CHOSEN
data_test = pd.read_json("processed_test.json")

In [5]:
#Retrains Dov2Vec model
taggedDocuments_train.extend(taggedDocuments_val)
model = Doc2Vec(vector_size=80, min_count=2, epochs=15)
model.build_vocab(taggedDocuments_train)
model.train(taggedDocuments_train, total_examples=model.corpus_count, epochs=model.epochs)

In [12]:
# Taggeddocument for test data
taggedDocuments_test = []
for index, data_point_post in data_test.iterrows():
    taggedDocument_test =  TaggedDocument(words=data_test.at[index, 'posts'], tags=[data_test.at[index, 'type']])
    taggedDocuments_test.append(taggedDocument_test)
    
type_test, posts_test = data_splitter(model, taggedDocuments_test)
type_train, posts_train = data_splitter(model, taggedDocuments_train)

In [15]:
# Applying logistic regression
logreg = LogisticRegression(n_jobs=-1, solver='sag')
logreg.fit((posts_train), (type_train))

# Testing the (logreg) model
type_pred = logreg.predict(posts_test)
print(classification_report(type_test, type_pred))

              precision    recall  f1-score   support

           E       0.87      0.85      0.86      2514
           I       0.86      0.88      0.87      2564

    accuracy                           0.87      5078
   macro avg       0.87      0.87      0.87      5078
weighted avg       0.87      0.87      0.87      5078



In [38]:
# Dummy classifier as a baseline to prove our model's effectiveness
dummy = DummyClassifier(strategy='uniform', random_state=0)
dummy.fit(posts_train+posts_val, type_train+type_val)

type_pred = dummy.predict(posts_test)
print(classification_report(type_test, type_pred))

0.505513981882631
              precision    recall  f1-score   support

           E       0.50      0.50      0.50      2488
           I       0.52      0.51      0.51      2590

    accuracy                           0.51      5078
   macro avg       0.51      0.51      0.51      5078
weighted avg       0.51      0.51      0.51      5078

